## Example: Converting a Segmentation Stack for Use with TrackGardener

This notebook provides a practical example of converting a segmentation stack - where each object’s value corresponds to its unique TrackID - to a TrackGardener database using the configuration file `Fluo-N2DL-HeLa-01_config_trackID.yaml`.

In this workflow, relationships between parent and offspring tracks are inferred based on their proximity in time and space. This approach is generally error-prone and not recommended for most applications. For more accurate lineage relationships, use an explicit graph structure; see labels_and_geff_to_TrackGardener for details of importing the graph structure from the `geff` format.

You can reuse the configuration file demonstrated here with the TrackGardener plugin itself. Just be sure to specify the correct paths to your imaging dataset and the TrackGardener database within the config file. Using absolute (rather than relative as for the purpose of this example) paths is the safest choice to ensure your files are correctly located.

In [9]:
import dask.array as da
import yaml

from track_gardener.converters.track_array_2_gardener import (
    assign_parent_offspring_relationships,
    convert_array_segmentations_to_db,
)

In [10]:
# pathways to the segmentation stack and configuration file

segm_path = './Fluo-N2DL-HeLa-01_segm.zarr/labels'
config_path = './Fluo-N2DL-HeLa-01_config_trackID.yaml'

In [11]:
# read segmentation array
segm_array = da.from_zarr(segm_path)

In [12]:
# read in configuration file
with open(config_path) as file:
    config = yaml.safe_load(file)

In [13]:
# convert and save to Track Gardener database
convert_array_segmentations_to_db(segm_array, config)

2025-07-27 20:17:31.200 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 2 labeled objects at t=0


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: cells.track_id, cells.t, cells.bbox_0, cells.bbox_1, cells.bbox_2, cells.bbox_3
[SQL: INSERT INTO cells (track_id, t, id, "row", col, bbox_0, bbox_1, bbox_2, bbox_3, mask, signals, tags) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: [(204, 0, 0, 83, 51, 70, 37, 98, 69, <memory at 0x0000020ED29F99C0>, '{"area": 693.0, "ch0_nuc": 154.99711399711398}', '{}'), (225, 0, 1, 72, 98, 58, 84, 89, 113, <memory at 0x0000020ED29FA140>, '{"area": 663.0, "ch0_nuc": 160.0663650075415}', '{}')]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [14]:
# guess parent-offspring relationships by space-time proximity
assign_parent_offspring_relationships(db_path=config['database']['path'], parent_radius=100)

2025-07-27 20:17:34.653 | INFO     | track_gardener.converters.track_array_2_gardener:assign_parent_offspring_relationships:172 - Connecting to database at Fluo-N2DL-HeLa-01_conversion_result.db
2025-07-27 20:17:34.660 | INFO     | track_gardener.converters.track_array_2_gardener:assign_parent_offspring_relationships:182 - Found 0 candidate offspring tracks
2025-07-27 20:17:34.662 | SUCCESS  | track_gardener.converters.track_array_2_gardener:assign_parent_offspring_relationships:249 - Parent-offspring assignment complete.
